In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../3.feature_extraction/features.tsv', sep='\t')
data.set_index('ID', inplace=True)
data = data.where(pd.notnull(data), None)
data.head()

,TITLE,BRAND,MODEL,RAM,STORAGE,PLUS,COLOR,SCREEN_SIZE
ID,,,,,,,,
7923488,Smartphone ASUS Zenfone II ZE551 16GB 4G Tela ...,asus,None,None,16,False,None,None
10026591,Caminhão Super Bombeiro Resgate Som E Luz - Ma...,None,None,None,None,False,None,None
10027542,Transformers 4 Power Battlers Dinobot Strafe -...,None,None,None,None,False,None,None
10029798,Violão Elétrico Michael Vm921 Folk Aço Natural...,None,None,None,None,False,None,None
10029845,Violão Elétrico Michael Venetian Vm625dtst Aço...,None,None,None,None,False,None,None


In [3]:
# True se algum dos dois valores for None OU os valores forem diferentes.
# Usados para comparação de campos "obrigatórios" para o match, como BRAND.
def either_absent_or_different(entry_1, entry_2, columns):
    for col in columns:
        col_e1 = entry_1[col]
        col_e2 = entry_2[col]
        if col_e1 is None or col_e2 is None:
            yield True
        else:
            yield col_e1 != col_e2


# True se ambos os valores estiverem presentes e forem o mesmo.
# Usado para comparação de specs como memória e armazenamento.
def both_present_and_same(entry_1, entry_2, columns):
    for col in columns:
        col_e1 = entry_1[col]
        col_e2 = entry_2[col]
        if col_e1 is None or col_e2 is None:
            yield False
        else:
            yield col_e1 == col_e2


# True se ambos os valores estiverem presentes e não forem o mesmo.
# Usado para encontrar diferenças de specs conhecidos.
def both_present_and_different(entry_1, entry_2, columns):
    for col in columns:
        col_e1 = entry_1[col]
        col_e2 = entry_2[col]
        if col_e1 is None or col_e2 is None:
            yield False
        else:
            yield col_e1 != col_e2

## Uncertainty
Medida de incerteza do match. Definida como o número de campos
None em pelo menos uma das instâncias.  
Mesmo que o resultado seja um match, pode ser que as informações
disponíveis apenas não tenham sido suficientes para detectar uma diferença
com certeza.  
Um valor maior dessa incerteza implica numa maior probabilidade de que
o match seja devido a falta de informações e, portanto, um match falso.

Dessa forma, um outro algoritmo mais adiante no processo pode considerar
apenas matches com nível mais baixo de incerteza.

In [4]:
def uncertainty(entry_1, entry_2):
    relevant_cols = ['BRAND', 'MODEL', 'RAM', 'STORAGE', 'SCREEN_SIZE']
    entry_1 = entry_1[relevant_cols]
    entry_2 = entry_2[relevant_cols]
    
    e1_nulls = entry_1.isnull()
    e2_nulls = entry_2.isnull()
    
    either_nulls = e1_nulls | e2_nulls
    
    return np.sum(either_nulls)

In [5]:
def match(entry_1, entry_2):
    spec_cols = ['RAM', 'STORAGE', 'SCREEN_SIZE']
    uncert = uncertainty(entry_1, entry_2)
    specs_in_common = both_present_and_same(entry_1, entry_2, spec_cols)
    
    if entry_1.PLUS != entry_2.PLUS:
        return False, uncert
    elif any(either_absent_or_different(entry_1, entry_2, ['BRAND'])):
        return False, uncert
    elif entry_1.MODEL is None or entry_2.MODEL is None:
        # Se o modelo não foi identificado, assumir que são o mesmo se e somente se
        # todos os specs existirem e forem os mesmos
        return all(specs_in_common), uncert
    elif entry_1.MODEL != entry_2.MODEL:
        return False, uncert
    elif any(both_present_and_different(entry_1, entry_2, spec_cols)):
        return False, uncert
    else:
        return True, uncert


def match_ids(id_1, id_2):
    return match(data.loc[id_1], data.loc[id_2])

In [6]:
from itertools import combinations

In [7]:
# Biblioteca para barra de progresso.
# O algorítmo é O(n^2), então pode demorar algum tempo.
# Se não quiserem ou não puderem instalar a biblioteca, basta remover este import
# e trocar o for por:
#    for pair in pairs:
from tqdm.autonotebook import tqdm

def find_matches():
    pairs = combinations(data.index, 2)
    n_items = len(data)
    n_pairs = (n_items * (n_items - 1)) // 2
    for pair in tqdm(pairs, total=n_pairs, desc='Finding matches...'):
        pair_match, dist = match_ids(*pair)
        if pair_match:
            yield pair, dist

C:\Users\wqfer\Anaconda3\envs\ml\lib\site-packages\tqdm\autonotebook\__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [8]:
matches = pd.DataFrame(find_matches(), columns=['PAIR', 'UNCERTAINTY'])

In [9]:
from operator import itemgetter

ids = matches.PAIR.apply([
    itemgetter(0),
    itemgetter(1)
])
matches.insert(0, 'ID1', ids.iloc[:, 0])
matches.insert(1, 'ID2', ids.iloc[:, 1])
matches.insert(2, 'TITLE1', data.loc[matches.ID1].reset_index().TITLE)
matches.insert(3, 'TITLE2', data.loc[matches.ID2].reset_index().TITLE)
matches.drop('PAIR', inplace=True, axis=1)

In [10]:
matches.sample(10)

,ID1,ID2,TITLE1,TITLE2,UNCERTAINTY
474,1025607795,1054895118,Apple Iphone 8 Plus 64gb Anatel,Iphone 8 Plus 64gb + Nota Fiscal + Lacrado (a...,2
54,11811144,1025208041,Smartphone Motorola Moto Z2 Play GamePad Editi...,Motorola Xt1710 Moto Z2 Play Dual Chip 64gb Or...,2
722,1077147848,1077157052,Celular Xiaomi Mi A2 64gb 4g Ram C\nota Black ...,Celular Xiaomi Mi A2 64gb 4g Ram C\nota Black ...,2
424,1018571570,1028671459,Smartphone Motorola Moto G6 Play 32gb Índigo X...,Celular Motorola Moto G6 Play Índigo Android 8...,2
834,1081996918,1084340822,Smartphone Xiaomi Mi A2 4gb/64gb Dual Sim Tela...,Smartphone Xiaomi Mi A2 Dual Sim 32g - 5.99 ...,3
130,849470172,1046787831,Celular Motorola Moto G6 Play Xt1922 4g 32gb D...,Motorola Moto G6 Play Xt1922 Nf Lacrado,3
465,1025208041,1029075911,Motorola Xt1710 Moto Z2 Play Dual Chip 64gb Or...,Celular Smartphone Motorola Moto Z2 Play 64gb ...,2
827,1081323151,1081331222,Smartphone Xiaomi Mi A2 Dualsim 4gb/32gb 5.99'...,Smartphone Xiaomi Mi A2 Dualsim 4gb/32gb 5.99'...,1
810,1080788081,1084468558,Smartphone Xiaomi Mi A2 4gb/64gb Dual Sim Tela...,Xiaomi Mi A2 Lite 64gb/4gb Ram V. Global Sem J...,2
593,1043880948,1079377092,Celular Samsung Galaxy A8 Plus 6 64gb 4g 16mp,Celular Samsung Galaxy A8 Plus 2018 Sm-a730f 6...,2


In [11]:
matches.to_csv('matches.tsv', sep='\t', index=False)